In [ ]:
import requests
import json
from openai import OpenAI
client = OpenAI()

In [19]:
### Get DataSet

In [20]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
### Index DataSet For Search

In [22]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [23]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [24]:
### Only FAQ Prompt

In [25]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [11]:
### LLM Call

In [26]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [27]:
### Full RAG

In [28]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
rag("can i still join the course")

'Yes, you can still join the course even after the start date. You are eligible to submit your homework, but be mindful of the deadlines for the final projects. Make sure not to leave things until the last minute.'

In [30]:
## Agentic RAG

In [66]:
prompt_template = """
You're a course teaching assistant for our courses.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [59]:
def initial_search(query,context="EMPTY"):
    prompt = prompt_template.format(question=query, context=context)
    print(prompt)
    
    answer = llm(prompt)
    print(answer)

In [60]:
initial_search("how do I run docker on gentoo?")

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
how do I run docker on gentoo?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}
{
"action": "ANSWER",
"answer": "To run Docker on Gentoo, you need to follow these steps: \n\n1. **Install Docker**: You can install Docker from the Gentoo repository. Use the following command:\n   ```bash\n   emerge app-containers/docker\n   ```

In [61]:
initial_search("how do I join the course?")

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
how do I join the course?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}
{
"action": "ANSWER",
"answer": "To join the course, you typically need to register through the course's official website or learning platform. Look for a 'Sign Up' or 'Enroll' button and follow the instructions provided. If there are prerequisites or 

In [62]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    return context.strip()

In [63]:
question = "how do I join the course?"
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
how do I join the course?
</QUESTION>

<CONTEXT> 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course start

In [64]:
answer = llm(prompt)
print(answer)

{
"action": "ANSWER",
"answer": "To join the course, you need to register using the provided link before the course starts. Although you can submit homeworks even if you registered late, make sure to follow the deadlines for the final projects. Additionally, join the course Telegram channel for announcements and subscribe to the course public Google Calendar for updates.",
"source": "CONTEXT"
}


In [67]:
initial_search("how do I join the course?")

You're a course teaching assistant for our courses.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
how do I join the course?
</QUESTION>

<CONTEXT> 
EMPTY
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}

If you can answer the QUESTION using CONTEXT, use this template:

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}

If the context doesn't contain the answer, use your own knowledge to answer the question

{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}
{
"action": "SEARCH",
"reasoning": "The CONTEXT is empty, so I need to look up the FAQ database for information about how to join the course."
}


In [68]:
def agentic_rag_v1(question):
    context = "EMPTY"
    prompt = prompt_template.format(question=question, context=context)
    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(answer)

    if answer['action'] == 'SEARCH':
        print('need to perform search...')
        search_results = search(question)
        context = build_context(search_results)
        
        prompt = prompt_template.format(question=question, context=context)
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(answer)

    return answer

In [71]:
agentic_rag_v1('how do I join the course?')

{'action': 'SEARCH', 'reasoning': 'The question asks how to join the course, but there is currently no context provided that specifically addresses this inquiry. Therefore, I will look up the relevant information in the FAQ database to provide an accurate response.'}
need to perform search...
{'action': 'ANSWER', 'answer': "To join the course, you need to register before the course starts. You can use the registration link provided in the course materials. Additionally, it’s a good idea to join the course Telegram channel for announcements and subscribe to the course public Google Calendar to stay updated. Even after the course starts, you're eligible to submit homework, but be mindful of deadlines for final projects.", 'source': 'CONTEXT'}


{'action': 'ANSWER',
 'answer': "To join the course, you need to register before the course starts. You can use the registration link provided in the course materials. Additionally, it’s a good idea to join the course Telegram channel for announcements and subscribe to the course public Google Calendar to stay updated. Even after the course starts, you're eligible to submit homework, but be mindful of deadlines for final projects.",
 'source': 'CONTEXT'}

In [72]:
agentic_rag_v1('how patch KDE under FreeBSD?')

{'action': 'ANSWER', 'answer': "Patching KDE under FreeBSD typically involves downloading the patches you want to apply, navigating to the source directory of your KDE installation, and then applying the patches using the `patch` command. You can find the source code for KDE in the ports collection for FreeBSD. The basic steps are as follows:\n\n1. Ensure you have the FreeBSD ports tree and that you're working with the correct version of the KDE source code.\n2. Download the necessary patch files from the KDE repository or the relevant source.\n3. Navigate to the source directory where KDE is installed (usually under `/usr/ports/x11/kde` or similar).\n4. Apply the patches using the command `patch < /path/to/your/patchfile.patch`.\n5. After applying, compile the software using the `make` and `make install` commands to finalize the changes.\n6. Once installed, check that KDE is running correctly with the patched changes.\n\nMake sure to read the patch's documentation for any additional i

{'action': 'ANSWER',
 'answer': "Patching KDE under FreeBSD typically involves downloading the patches you want to apply, navigating to the source directory of your KDE installation, and then applying the patches using the `patch` command. You can find the source code for KDE in the ports collection for FreeBSD. The basic steps are as follows:\n\n1. Ensure you have the FreeBSD ports tree and that you're working with the correct version of the KDE source code.\n2. Download the necessary patch files from the KDE repository or the relevant source.\n3. Navigate to the source directory where KDE is installed (usually under `/usr/ports/x11/kde` or similar).\n4. Apply the patches using the command `patch < /path/to/your/patchfile.patch`.\n5. After applying, compile the software using the `make` and `make install` commands to finalize the changes.\n6. Once installed, check that KDE is running correctly with the patched changes.\n\nMake sure to read the patch's documentation for any additional 

In [73]:
### Agentic Search

In [81]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [82]:
question = "how do I join the course?"

search_queries = []
search_results = []
previous_actions = []
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=3,
    iteration_number=1
)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number

In [83]:
answer_json = llm(prompt)
answer = json.loads(answer_json)
print(json.dumps(answer, indent=2))

{
  "action": "SEARCH",
  "reasoning": "I need to find information about the process for joining the course as the current context does not contain this information.",
  "keywords": [
    "how to join the course",
    "enrollment process",
    "course registration"
  ]
}


In [84]:
previous_actions.append(answer)

In [86]:
keywords = answer['keywords']
search_queries.extend(keywords)

for k in keywords:
    res = search(k)
    search_results.extend(res)

In [88]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

search_results = dedup(search_results)

In [89]:
def agentic_search(question):
    search_queries = []
    search_results = []
    previous_actions = []

    iteration = 0
    
    while True:
        print(f'ITERATION #{iteration}...')
    
        context = build_context(search_results)
        prompt = prompt_template.format(
            question=question,
            context=context,
            search_queries="\n".join(search_queries),
            previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
            max_iterations=3,
            iteration_number=iteration
        )
    
        print(prompt)
    
        answer_json = llm(prompt)
        answer = json.loads(answer_json)
        print(json.dumps(answer, indent=2))

        previous_actions.append(answer)
    
        action = answer['action']
        if action != 'SEARCH':
            break
    
        keywords = answer['keywords']
        search_queries = list(set(search_queries) | set(keywords))

        for k in keywords:
            res = search(k)
            search_results.extend(res)
    
        search_results = dedup(search_results)
        
        iteration = iteration + 1
        if iteration >= 4:
            break
    
        print()

    return answer

In [90]:
agentic_search('how do I prepare for the course?')

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

{'action': 'ANSWER',
 'answer': "To prepare for the course, consider the following strategies:\n1. **Register early:** Make sure to register for the course before it begins.\n2. **Join communication channels:** Subscribe to the public Google Calendar for schedule updates, and join the course's Telegram channel and DataTalks.Club's Slack for announcements and peer discussions.\n3. **Familiarize yourself with tools:** If the course uses Git or GitHub, set up your account and understand how to clone repositories. This will help you access course materials.\n4. **Plan your study schedule:** Allocate specific times each week to study course materials, complete assignments, and engage with peers. Having a structured schedule can enhance your learning experience.\n5. **Prepare for hands-on work:** If applicable, ensure you have the necessary tools and software set up on your computer to participate in practical assignments.\n6. **Review course materials:** If any preliminary materials or read

In [91]:
### Function As Tools

In [100]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [101]:
tools = [search_tool]

In [102]:
def do_call(tool_call_response):
    function_name = tool_call_response.name
    arguments = json.loads(tool_call_response.arguments)

    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.call_id,
        "output": json.dumps(result, indent=2),
    }

In [103]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.
When using FAQ, perform deep topic exploration: make one request to FAQ,
and then based on the results, make more requests.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

In [104]:
while True: # main Q&A loop
    question = input() # How do I do my best for module 1?
    if question == 'stop':
        break

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True: # request-response loop - query API till get a message
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_tool_calls = False
        
        for entry in response.output:
            chat_messages.append(entry)
        
            if entry.type == 'function_call':      
                print('function_call:', entry)
                print()
                result = do_call(entry)
                chat_messages.append(result)
                has_tool_calls = True

            elif entry.type == 'message':
                print(entry.content[0].text)
                print()

        if not has_tool_calls:
            break

 get the best points in all modules?


function_call: ResponseFunctionToolCall(arguments='{"query":"best points in all modules"}', call_id='call_admAh3U3dIdgfNvTRcuXowRF', name='search', type='function_call', id='fc_6884f4c360ec819bb057b46ad90585be0ddef9a7325ed654', status='completed')

To get the best points in all modules of the course, here's a general overview of how the points system works:

1. **Homework Grading**: Your homework submissions are graded based on the number of questions in each assignment. You can view the points earned directly on the homework page.

2. **Leaderboards**: The total points you've earned, including those from homework, FAQs submitted, and "Learning in Public" links, are displayed on the leaderboard.

3. **Specific Module Points**: Each module may have its own unique grading criteria. It's crucial to thoroughly engage with the content, follow instructions, and possibly contribute to discussions or FAQs for maximum points.

If you're looking for specific strategies to maximize points in cert

 stop


In [109]:
from IPython.display import display, HTML
import markdown # pip install markdown
    

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_messages = [
    {"role": "developer", "content": developer_prompt},
]

# Chat loop
while True:
    question = input()
    
    if question.strip().lower() == 'stop':
        print("Chat ended.")
        break
    print()

    message = {"role": "user", "content": question}
    chat_messages.append(message)

    while True:  # inner request loop
        response = client.responses.create(
            model='gpt-4o-mini',
            input=chat_messages,
            tools=tools
        )

        has_messages = False

        for entry in response.output:
            chat_messages.append(entry)

            if entry.type == "function_call":
                result = do_call(entry)
                chat_messages.append(result)
                display_function_call(entry, result)

            elif entry.type == "message":
                display_response(entry)
                has_messages = True

        if has_messages:
            break

 get the best points in all modules


NameError: name 'display_function_call' is not defined

In [112]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

'wget' is not recognized as an internal or external command,
operable program or batch file.
